For Predictions of Simulated land use plans.
These go directly into feature stack no need to crop and mask

# Imports

In [6]:
import os
import sys
import math
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import fiona
import pandas as pd
import geopandas as gpd
from rasterio import features
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show
from scipy.ndimage import distance_transform_edt


In [7]:
# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)

In [8]:
from constants import TREECOVER_2010, OUTPUT_PATH, ROAD_PATH , RIVER_PATH, SIMULATION_FEATURES_DIR

# Pixel Distance to Nearest Road

Here we determine the distance to Roads and Rivers within the land use plans.

In [9]:
lup_raster_25 = rasterio.open(os.path.join(SIMULATION_FEATURES_DIR[0],'sim25', 'sim25_raster.tif'))
# Read the raster data into a NumPy array
lup_raster_array_25 = lup_raster_25.read(1)


In [12]:
lup_raster_50 = rasterio.open(os.path.join(SIMULATION_FEATURES_DIR[0],'sim50', 'sim50_raster.tif'))
# Read the raster data into a NumPy array
lup_raster_array_50 = lup_raster_50.read(1)

In [ ]:
lup_raster_5 = rasterio.open(os.path.join(SIMULATION_FEATURES_DIR[0], 'sim5', 'sim5_raster.tif'))
# Read the raster data into a NumPy array
lup_raster_array_5 = lup_raster_5.read(1)

In [ ]:
lup_raster_hedges = rasterio.open(os.path.join(SIMULATION_FEATURES_DIR[0], 'simhedges','simhedges_raster.tif'))
# Read the raster data into a NumPy array
lup_raster_array_hedges = lup_raster_hedges.read(1)

In [ ]:
# Read in road vector
vector = gpd.read_file(ROAD_PATH[0]).to_crs("EPSG:4326")

# Get list of geometries for all features in vector file
geom = [shapes for shapes in vector.geometry]



In [11]:
lup_raster_25.crs

CRS.from_epsg(4326)

In [10]:
# Open example raster for profile details
raster = rasterio.open(TREECOVER_2010)

#

In [11]:
# Rasterize road vector using the shape and coordinate system of the raster
rasterized = features.rasterize(geom,
                                out_shape = raster.shape,
                                fill = -1,
                                out = None,
                                transform = raster.transform,
                                all_touched = False,
                                default_value = 1,
                                dtype = None)



In [12]:
# Modify the nodata value and the data type in the raster profile
profile = raster.profile
profile.update(nodata=-1, dtype='float32')

profile


{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -1, 'width': 20381, 'height': 22512, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, -62.64186038139295,
       0.0, -0.00026949458523585647, -19.287457970745013), 'tiled': False, 'interleave': 'band'}

In [13]:
# Where files will save, can add subfolders if desired
output_dir = os.path.join(OUTPUT_PATH[0], 'processed_rasters', 'road_raster_sims')
os.makedirs(output_dir, exist_ok=True)

# Create Distance to roads for Simulation

In [14]:
# Convert the rasterized array to boolean where True represents roads
road_mask = rasterized == 1

In [15]:
# Calculate the Euclidean distance from each road pixel to every other pixel
distances = distance_transform_edt(~road_mask)


# Process

### 25%

In [21]:


# Convert the lup_raster array to boolean where True represents the area of interest
area_mask = np.isin(lup_raster_array_25, [1, 2, 3, 4])

# For pixels outside the area of interest, set the distance to a nodata value
distances[~area_mask] = -1

output_filepath = os.path.join(output_dir, 'road_raster_sim25.tif')

# Write the rasterized data to a new GeoTIFF file
with rasterio.open(output_filepath, 'w', **profile) as dst:
    dst.write(distances.astype('float32'), 1)

### 50%

In [22]:
# Convert the lup_raster array to boolean where True represents the area of interest
area_mask = np.isin(lup_raster_array_50, [1, 2, 3, 4])


# For pixels outside the area of interest, set the distance to a nodata value
distances[~area_mask] = -1

output_filepath = os.path.join(output_dir, 'road_raster_sim50.tif')

# Write the rasterized data to a new GeoTIFF file
with rasterio.open(output_filepath, 'w', **profile) as dst:
    dst.write(distances.astype('float32'), 1)

### 5%

In [16]:
# Convert the lup_raster array to boolean where True represents the area of interest
area_mask = np.isin(lup_raster_array_5, [1, 2, 3, 4])


# For pixels outside the area of interest, set the distance to a nodata value
distances[~area_mask] = -1

output_filepath = os.path.join(output_dir, 'road_raster_sim5.tif')

# Write the rasterized data to a new GeoTIFF file
with rasterio.open(output_filepath, 'w', **profile) as dst:
    dst.write(distances.astype('float32'), 1)





### HEDGEROWS

In [17]:
# Convert the lup_raster array to boolean where True represents the area of interest
area_mask = np.isin(lup_raster_array_hedges, [1, 2, 3, 4])


# For pixels outside the area of interest, set the distance to a nodata value
distances[~area_mask] = -1

output_filepath = os.path.join(output_dir, 'road_raster_simhedges.tif')

# Write the rasterized data to a new GeoTIFF file
with rasterio.open(output_filepath, 'w', **profile) as dst:
    dst.write(distances.astype('float32'), 1)

# Pixel Distance to Nearest River

In [18]:
# Where files will save, can add subfolders if desired
output_dir = os.path.join(OUTPUT_PATH[0], 'processed_rasters', 'river_raster_sims')
os.makedirs(output_dir, exist_ok=True)

In [21]:
# Read in vector
vector_river = gpd.read_file(RIVER_PATH[0]).to_crs("EPSG:4326")

# Get list of geometries for all features in vector file
geom_river = [shapes for shapes in vector_river.geometry]


In [22]:
# Rasterize vector using the shape and coordinate system of the raster
rasterized_river = features.rasterize(geom_river,
                                out_shape = raster.shape,
                                fill = -1,
                                out = None,
                                transform = raster.transform,
                                all_touched = False,
                                default_value = 1,
                                dtype = None)

In [23]:
# Convert the rasterized array to boolean where True represents roads
river_mask = rasterized_river == 1
# Calculate the Euclidean distance from each road pixel to every other pixel
distances_river = distance_transform_edt(~river_mask)

In [24]:
# Convert the lup_raster array to boolean where True represents the area of interest
area_mask = np.isin(lup_raster_array_25, [1, 2, 3, 4])

# For pixels outside the area of interest, set the distance to a nodata value
distances_river[~area_mask] = -1

output_filepath = os.path.join(output_dir, 'river_raster-sim25.tif')

# Write the rasterized data to a new GeoTIFF file
with rasterio.open(output_filepath, 'w', **profile) as dst:
    dst.write(distances_river.astype('float32'), 1)

In [25]:

# Convert the lup_raster array to boolean where True represents the area of interest
area_mask = np.isin(lup_raster_array_5, [1, 2, 3, 4])

# For pixels outside the area of interest, set the distance to a nodata value
distances_river[~area_mask] = -1

output_filepath = os.path.join(output_dir, 'river_raster-sim5.tif')

# Write the rasterized data to a new GeoTIFF file
with rasterio.open(output_filepath, 'w', **profile) as dst:
    dst.write(distances_river.astype('float32'), 1)

In [26]:

# Convert the lup_raster array to boolean where True represents the area of interest
area_mask = np.isin(lup_raster_array_50, [1, 2, 3, 4])

# For pixels outside the area of interest, set the distance to a nodata value
distances_river[~area_mask] = -1

output_filepath = os.path.join(output_dir, 'river_raster-sim50.tif')

# Write the rasterized data to a new GeoTIFF file
with rasterio.open(output_filepath, 'w', **profile) as dst:
    dst.write(distances_river.astype('float32'), 1)

In [ ]:
# Convert the lup_raster array to boolean where True represents the area of interest
area_mask = np.isin(lup_raster_array_hedges, [1, 2, 3, 4])

# For pixels outside the area of interest, set the distance to a nodata value
distances_river[~area_mask] = -1

output_filepath = os.path.join(output_dir, 'river_raster-simhedges.tif')

# Write the rasterized data to a new GeoTIFF file
with rasterio.open(output_filepath, 'w', **profile) as dst:
    dst.write(distances_river.astype('float32'), 1)